<a href="https://colab.research.google.com/github/animesh-77/Active-Fire-Detection/blob/3algo/Models_and_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/animesh-77/Active-Fire-Detection.git

Cloning into 'Active-Fire-Detection'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 105 (delta 45), reused 80 (delta 36), pack-reused 0
Receiving objects: 100% (105/105), 21.36 MiB | 28.04 MiB/s, done.
Resolving deltas: 100% (45/45), done.


# Modeling different architectures

## 1. U-Net

### 1.1 10 channel inputs all filters

In [6]:
import tensorflow as tf
import keras

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose, Input, BatchNormalization

In [7]:


def build_model(input_layer, start_neurons):
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    batch1= BatchNormalization()(conv1)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(batch1)
    batch1= BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(batch1)
    pool1 = Dropout(0.25)(pool1)

    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    batch2= BatchNormalization()(conv2)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(batch2)
    batch2= BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(batch2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    batch3= BatchNormalization()(conv3)
    conv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(batch3)
    batch3= BatchNormalization()(conv3)
    pool3 = MaxPooling2D((2, 2))(batch3)
    pool3 = Dropout(0.5)(pool3)

    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    batch4= BatchNormalization()(conv4)
    conv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(batch4)
    batch4= BatchNormalization()(conv4)
    pool4 = MaxPooling2D((2, 2))(batch4)
    pool4 = Dropout(0.5)(pool4)

    # Middle
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    batchm = BatchNormalization()(convm)
    convm = Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(batchm)
    batchm = BatchNormalization()(convm)
    
    deconv4 = Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(batchm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    ubatch4 = BatchNormalization()(uconv4)
    uconv4 = Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(ubatch4)
    ubatch4 = BatchNormalization()(uconv4)
    

    deconv3 = Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(ubatch4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    ubatch3 = BatchNormalization()(uconv3)
    uconv3 = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(ubatch3)
    ubatch3 = BatchNormalization()(uconv3)

    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(ubatch3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    ubatch2 = BatchNormalization()(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(ubatch2)
    ubatch2 = BatchNormalization()(uconv2)

    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(ubatch2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    ubatch1 = BatchNormalization()(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(ubatch1)
    ubatch1 = BatchNormalization()(uconv1)
    
    output_layer = Conv2D(1, (1,1), padding="same", activation="sigmoid")(ubatch1)
    
    return output_layer

img_size_target= 256


In [8]:
channels= 10
input_layer = Input((img_size_target, img_size_target, channels))
output_layer = build_model(input_layer, 64)

In [9]:
tf.compat.v1.reset_default_graph()

Unet_10c= tf.keras.Model(input_layer, output_layer, name= "10_channel")
Unet_10c.summary()

Model: "10_channel"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                0)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  5824        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                        

In [ ]:
## loading weights

Unet_10c.load_weights("models/intersection/unet_64f_2conv_10c/model_unet_Intersection_final_weights.h5")
